In [ ]:
# ==================== 最小实现：统一配置（只改这里） ====================

# ---- 服务配置（OpenAI-compatible）----
HOST_IP = "61.169.215.178"
LLM_PORT = 10000
EMBEDDING_PORT = 10004
RERANKER_PORT = 10005
API_KEY = "xtsk"

LLM_MODEL = "LLM"
EMBEDDING_MODEL = "EMBEDDING"
RERANKER_MODEL = "RERANKER"

# ---- pgvector 配置 ----
PG_HOST = "61.169.215.186"
PG_PORT = 15432
PG_DB = "vectordb"
PG_USER = "admin"
PG_PASSWORD = "xtsk@2024"

# ---- 向量维度（Qwen3 Matryoshka 截断）----
TARGET_DIM = 1536  # pgvector index 限制 2000 维

# ---- 运行参数 ----
INGEST_N_CASES = 5269
MAX_WORKERS = 5
LLM_TIMEOUT_S = 80
EMBED_TIMEOUT_S = 60
RERANK_TIMEOUT_S = 30

# ---- imports / clients ----
import json
import time
from datetime import datetime
from typing import Dict, List, Optional, TypedDict, Annotated, Sequence

import requests
import psycopg2
from openai import OpenAI

EMBEDDING_URL = f"http://{HOST_IP}:{EMBEDDING_PORT}"
RERANKER_URL = f"http://{HOST_IP}:{RERANKER_PORT}"
LLM_BASE_URL = f"http://{HOST_IP}:{LLM_PORT}/v1"

DB_CONFIG = {
    "host": PG_HOST,
    "port": PG_PORT,
    "database": PG_DB,
    "user": PG_USER,
    "password": PG_PASSWORD,
}

llm_client = OpenAI(api_key=API_KEY, base_url=LLM_BASE_URL)


def log(msg: str):
    print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")


log("✅ config loaded")
log(f"LLM={LLM_BASE_URL} model={LLM_MODEL} timeout={LLM_TIMEOUT_S}s")
log(f"EMB={EMBEDDING_URL} model={EMBEDDING_MODEL} truncate={TARGET_DIM} timeout={EMBED_TIMEOUT_S}s")
log(f"RERANK={RERANKER_URL} model={RERANKER_MODEL} timeout={RERANK_TIMEOUT_S}s")
log(f"PG={PG_HOST}:{PG_PORT}/{PG_DB}")
log(f"INGEST_N_CASES={INGEST_N_CASES} MAX_WORKERS={MAX_WORKERS}")

[17:16:17] ✅ config loaded
[17:16:17] LLM=http://61.169.215.178:10000/v1 model=LLM
[17:16:17] EMB=http://61.169.215.178:10004 model=EMBEDDING truncate=1536
[17:16:17] RERANK=http://61.169.215.178:10005 model=RERANKER
[17:16:17] PG=61.169.215.186:15432/vectordb


In [ ]:
# ==================== LLM 结构化抽取 ====================

with open("corpus/_all_cases.json", "r", encoding="utf-8") as f:
    all_cases = json.load(f)

STRUCTURE_PROMPT = """你是中医医案结构化专家。请将以下医案内容提取为标准 JSON。

只输出 JSON：
{
  \"patient\": {\"age\": int|null, \"sex\": \"男\"|\"女\"|null},
  \"chief_complaint\": str|null,
  \"tcm_diagnosis\": str|null,
  \"western_diagnosis\": str|null,
  \"tcm_pattern\": str|null,
  \"treatment_principle\": str|null,
  \"prescriptions\": [{\"visit\": str, \"herbs\": [{\"name\": str, \"dose\": str}], \"days\": int|null}],
  \"outcome\": str|null,
  \"summary\": str
}

字段为空用 null。

医案内容：
"""


def strip_code_fence(text: str) -> str:
    t = (text or "").strip()
    if t.startswith("```"):
        parts = t.split("```")
        if len(parts) >= 2:
            t = parts[1].strip()
            if t.startswith("json"):
                t = t[4:].strip()
    return t


def extract_case_structure(content: str, max_retries: int = 3) -> Optional[Dict]:
    for attempt in range(1, max_retries + 1):
        try:
            resp = llm_client.chat.completions.create(
                model=LLM_MODEL,
                messages=[
                    {"role": "system", "content": "只输出 JSON。"},
                    {"role": "user", "content": STRUCTURE_PROMPT + content[:5000]},
                ],
                temperature=0,
                timeout=LLM_TIMEOUT_S,
            )
            return json.loads(strip_code_fence(resp.choices[0].message.content))
        except Exception as e:
            log(f"LLM extract failed attempt={attempt}/{max_retries}: {str(e)[:120]}")
            if attempt < max_retries:
                time.sleep(2)
    return None


log(f"cases loaded: {all_cases.get('count')} total")
log(f"preview: {all_cases['cases'][0]['content'][:120]}...")

In [ ]:
# ==================== 建表 + 入库（前 N 条） ====================

from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock


def init_tables():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    cur.execute("DROP TABLE IF EXISTS tcm_case_embedding CASCADE;")
    cur.execute("DROP TABLE IF EXISTS tcm_case CASCADE;")

    cur.execute(
        """
        CREATE TABLE tcm_case (
            id SERIAL PRIMARY KEY,
            source_md TEXT,
            case_index INT,
            raw_content TEXT,
            clean_json JSONB,
            sex TEXT,
            age INT,
            tcm_pattern TEXT,
            tcm_diagnosis TEXT,
            western_diagnosis TEXT,
            summary TEXT,
            created_at TIMESTAMP DEFAULT NOW()
        );
        """
    )

    cur.execute(
        f"""
        CREATE TABLE tcm_case_embedding (
            id SERIAL PRIMARY KEY,
            case_id INT REFERENCES tcm_case(id) ON DELETE CASCADE,
            view TEXT,
            content TEXT,
            embedding vector({TARGET_DIM}),
            created_at TIMESTAMP DEFAULT NOW()
        );
        """
    )

    cur.execute("CREATE INDEX idx_embedding_view ON tcm_case_embedding(view);")

    cur.execute(
        """
        CREATE INDEX idx_embedding_vector
        ON tcm_case_embedding
        USING hnsw (embedding vector_cosine_ops);
        """
    )

    conn.commit()
    cur.close()
    conn.close()


def embed_truncated(text: str, max_retries: int = 3) -> Optional[List[float]]:
    url = f"{EMBEDDING_URL}/v1/embeddings"
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {API_KEY}"}
    payload = {"model": EMBEDDING_MODEL, "input": [text]}

    for attempt in range(1, max_retries + 1):
        try:
            r = requests.post(url, headers=headers, json=payload, timeout=EMBED_TIMEOUT_S)
            r.raise_for_status()
            full = r.json()["data"][0]["embedding"]
            return full[:TARGET_DIM]
        except Exception as e:
            if attempt == max_retries:
                log(f"embedding failed: {str(e)[:120]}")
                return None
            time.sleep(1)


def process_one_case(case_data: Dict, idx: int, total: int) -> bool:
    log(f"[{idx}/{total}] start {case_data['source_md']}#{case_data['index']}")

    t0 = time.time()
    structured = extract_case_structure(case_data["content"], max_retries=3)
    t_llm = time.time() - t0

    if not structured:
        log(f"[{idx}/{total}] LLM failed ({t_llm:.1f}s)")
        return False

    log(f"[{idx}/{total}] LLM ok ({t_llm:.1f}s)")

    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    cur.execute(
        """
        INSERT INTO tcm_case (source_md, case_index, raw_content, clean_json, sex, age, tcm_pattern, tcm_diagnosis, western_diagnosis, summary)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        RETURNING id;
        """,
        (
            case_data["source_md"],
            case_data["index"],
            case_data["content"],
            json.dumps(structured, ensure_ascii=False),
            (structured.get("patient") or {}).get("sex"),
            (structured.get("patient") or {}).get("age"),
            structured.get("tcm_pattern"),
            structured.get("tcm_diagnosis"),
            structured.get("western_diagnosis"),
            structured.get("summary"),
        ),
    )
    case_id = cur.fetchone()[0]

    views = [
        ("summary", structured.get("summary") or ""),
        ("pattern", f"{structured.get('tcm_pattern') or ''} {structured.get('treatment_principle') or ''}".strip()),
    ]

    for view, content in views:
        if not content:
            continue
        t1 = time.time()
        emb = embed_truncated(content[:500], max_retries=3)
        t_emb = time.time() - t1
        if emb is None:
            log(f"[{idx}/{total}] embedding failed view={view} ({t_emb:.1f}s)")
            continue
        cur.execute(
            """INSERT INTO tcm_case_embedding (case_id, view, content, embedding) VALUES (%s,%s,%s,%s);""",
            (case_id, view, content, emb),
        )
        log(f"[{idx}/{total}] embedding ok view={view} ({t_emb:.1f}s)")

    conn.commit()
    cur.close()
    conn.close()

    log(f"[{idx}/{total}] done case_id={case_id}\n")
    return True


init_tables()

cases_to_ingest = all_cases["cases"][:INGEST_N_CASES]
log(f"ingest N={len(cases_to_ingest)} workers={MAX_WORKERS}")

ok = 0
fail = 0
lock = Lock()

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    futures = [ex.submit(process_one_case, c, i + 1, len(cases_to_ingest)) for i, c in enumerate(cases_to_ingest)]
    for fu in as_completed(futures):
        try:
            success = fu.result()
        except Exception as e:
            success = False
            log(f"task exception: {str(e)[:120]}")

        with lock:
            ok += 1 if success else 0
            fail += 0 if success else 1

log(f"ingest done success={ok} fail={fail}")

In [ ]:
# ==================== RAG 检索（metadata + vector + rerank） ====================

class TCMCaseRAG:
    def _query_embedding(self, text: str) -> List[float]:
        url = f"{EMBEDDING_URL}/v1/embeddings"
        headers = {"Content-Type": "application/json", "Authorization": f"Bearer {API_KEY}"}
        payload = {"model": EMBEDDING_MODEL, "input": [text]}
        r = requests.post(url, headers=headers, json=payload, timeout=EMBED_TIMEOUT_S)
        r.raise_for_status()
        full = r.json()["data"][0]["embedding"]
        return full[:TARGET_DIM]

    def _rerank(self, query: str, candidates: List[Dict], top_k: int) -> List[Dict]:
        url = f"{RERANKER_URL}/v1/rerank"
        headers = {"Content-Type": "application/json", "Authorization": f"Bearer {API_KEY}"}
        docs = [c.get("summary") or c.get("content", "")[:300] for c in candidates]
        payload = {"model": RERANKER_MODEL, "query": query, "documents": docs, "top_k": min(top_k, len(docs))}
        r = requests.post(url, headers=headers, json=payload, timeout=RERANK_TIMEOUT_S)
        r.raise_for_status()
        out = []
        for item in r.json()["results"]:
            idx = item["index"]
            c = candidates[idx]
            c["rerank_score"] = item["relevance_score"]
            out.append(c)
        return out

    def search(self, query: str, filters: Optional[Dict] = None, top_k: int = 10, rerank_top_k: int = 5, view: str = "summary") -> List[Dict]:
        qv = self._query_embedding(query)

        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()

        sql = """
            SELECT
                c.id, c.source_md, c.case_index, c.summary, c.sex, c.age, c.tcm_pattern,
                c.tcm_diagnosis, c.western_diagnosis, c.clean_json, e.content,
                1 - (e.embedding <=> %s::vector) AS similarity
            FROM tcm_case c
            JOIN tcm_case_embedding e ON c.id = e.case_id
            WHERE e.view = %s
        """
        params = [qv, view]

        if filters:
            if filters.get("sex"):
                sql += " AND c.sex = %s"
                params.append(filters["sex"])
            if filters.get("age_min") is not None:
                sql += " AND c.age >= %s"
                params.append(filters["age_min"])
            if filters.get("age_max") is not None:
                sql += " AND c.age <= %s"
                params.append(filters["age_max"])
            if filters.get("tcm_pattern"):
                sql += " AND c.tcm_pattern ILIKE %s"
                params.append(f"%{filters['tcm_pattern']}%")

        sql += " ORDER BY similarity DESC LIMIT %s;"
        params.append(top_k)

        cur.execute(sql, params)
        rows = cur.fetchall()
        cur.close()
        conn.close()

        candidates = []
        for r in rows:
            candidates.append(
                {
                    "case_id": r[0],
                    "source_md": r[1],
                    "case_index": r[2],
                    "summary": r[3],
                    "sex": r[4],
                    "age": r[5],
                    "tcm_pattern": r[6],
                    "tcm_diagnosis": r[7],
                    "western_diagnosis": r[8],
                    "clean_json": r[9],
                    "content": r[10],
                    "similarity": float(r[11]),
                }
            )

        if not candidates:
            return []

        return self._rerank(query, candidates, rerank_top_k)


rag = TCMCaseRAG()

q = "失眠多梦，心烦易怒，如何治疗？"
res = rag.search(q, top_k=5, rerank_top_k=3)
print("RAG test:", q)
for i, r in enumerate(res, 1):
    print(i, r["source_md"], r["case_index"], f"sim={r['similarity']:.3f}", f"rerank={r.get('rerank_score', 0):.3f}")

In [ ]:
# ==================== LangGraph Agent（parse → retrieve → generate） ====================

from langgraph.graph import StateGraph, END
import operator


class AgentState(TypedDict):
    query: str
    parsed_query: dict
    retrieved_cases: list
    answer: str
    messages: Annotated[Sequence[str], operator.add]


def parse_query_node(state: AgentState) -> AgentState:
    query = state["query"]

    prompt = f"""请把用户问题解析成检索用 JSON。只输出 JSON：
{{
  \"filters\": {{\"sex\": \"男\"|\"女\"|null, \"age_min\": int|null, \"age_max\": int|null, \"tcm_pattern\": str|null}},
  \"rewritten_query\": str
}}

用户问题: {query}
"""

    try:
        resp = llm_client.chat.completions.create(
            model=LLM_MODEL,
            messages=[
                {"role": "system", "content": "只输出 JSON。"},
                {"role": "user", "content": prompt},
            ],
            temperature=0,
            timeout=LLM_TIMEOUT_S,
        )
        parsed = json.loads(strip_code_fence(resp.choices[0].message.content))
        state["parsed_query"] = parsed
        state["messages"].append(f"parse: {parsed}")
    except Exception as e:
        state["parsed_query"] = {"filters": {}, "rewritten_query": query}
        state["messages"].append(f"parse_failed: {str(e)[:80]}")

    return state


def retrieve_cases_node(state: AgentState) -> AgentState:
    parsed = state.get("parsed_query") or {}
    filters = {k: v for k, v in (parsed.get("filters") or {}).items() if v is not None}
    rewritten = parsed.get("rewritten_query") or state["query"]

    try:
        results = rag.search(rewritten, filters=filters or None, top_k=8, rerank_top_k=5)
        state["retrieved_cases"] = results
        state["messages"].append(f"retrieve: {len(results)} cases")
    except Exception as e:
        state["retrieved_cases"] = []
        state["messages"].append(f"retrieve_failed: {str(e)[:80]}")

    return state


def generate_answer_node(state: AgentState) -> AgentState:
    query = state["query"]
    cases = state.get("retrieved_cases") or []

    if not cases:
        state["answer"] = "没有检索到相关医案。"
        state["messages"].append("generate: no cases")
        return state

    ctx = ""
    for i, c in enumerate(cases[:3], 1):
        ctx += f"\n[医案{i}] {c['source_md']}#{c['case_index']} | {c.get('sex')}/{c.get('age')} | {c.get('tcm_pattern')}\n"
        ctx += f"摘要: {c.get('summary','')}\n"

    prompt = f"""你是中医专家。基于检索到的医案回答用户问题。

用户问题: {query}

检索到的医案摘要:
{ctx}

要求:
- 结合医案给出辨证思路、治法和用药方向
- 引用医案编号
- 300字内

回答:"""

    try:
        resp = llm_client.chat.completions.create(
            model=LLM_MODEL,
            messages=[
                {"role": "system", "content": "你是中医专家。"},
                {"role": "user", "content": prompt},
            ],
            temperature=0.3,
            timeout=LLM_TIMEOUT_S,
        )
        state["answer"] = resp.choices[0].message.content.strip()
        state["messages"].append("generate: ok")
    except Exception as e:
        state["answer"] = f"生成失败: {str(e)[:120]}"
        state["messages"].append("generate_failed")

    return state


def build_agent():
    g = StateGraph(AgentState)
    g.add_node("parse", parse_query_node)
    g.add_node("retrieve", retrieve_cases_node)
    g.add_node("generate", generate_answer_node)

    g.set_entry_point("parse")
    g.add_edge("parse", "retrieve")
    g.add_edge("retrieve", "generate")
    g.add_edge("generate", END)
    return g.compile()


agent = build_agent()
print("✅ agent ready")

🔍 测试 RAG 检索

查询: 失眠多梦，心烦易怒，如何治疗？
召回 5 个医案:

1. [上海名老中医医案精选.md#7] 相似度:0.776 Rerank:0.884
   患者: 女/31岁 | 证型: 肝亢肾虚
   摘要: 主要症状为长期失眠、多梦、夜尿频急、头胀痛、颈部板滞、手麻、心烦紧张、腰酸腿麻；证型为肝亢肾虚；核心治法为平肝益肾安神；经加味龙牡汤合仙地汤加减治疗，二诊后睡眠明显改善，诸症减轻，疗效显著。...

2. [上海名老中医医案精选.md#9] 相似度:0.715 Rerank:0.875
   患者: 女/42岁 | 证型: 肝郁阳亢，气血痹阻
   摘要: 患者主要症状为长期失眠伴入睡困难、多梦、头晕头胀、口干、心慌、便秘等，证属肝郁阳亢、气血痹阻。核心治法为解郁平肝、通络活血、清热安神，方用甘麦苦参汤合加味柴胡龙牡汤加减。经治疗后睡眠明显改善，诸症减轻...

3. [上海名老中医医案精选.md#8] 相似度:0.732 Rerank:0.874
   患者: 女/24岁 | 证型: 肝木偏旺，瘀热交阻
   摘要: 主要症状为失眠、多梦易醒、心烦易怒、颈板不适；证型为肝木偏旺，瘀热交阻；核心治法为平肝抑木，清热化瘀；经三诊治疗后夜寐恢复正常，疗效显著。...

4. [上海名老中医医案精选.md#15] 相似度:0.745 Rerank:0.872
   患者: 男/69岁 | 证型: 肝阳上亢化风，瘀热交阻
   摘要: 主要症状为夜寐多梦、噩梦、说梦话、梦中紧张及踢打动作；证型为肝阳上亢化风，瘀热交阻；核心治法为平肝潜阳息风，清热活血安神；经28剂中药治疗后症状明显减轻，疗效显著。...

5. [上海名老中医医案精选.md#10] 相似度:0.723 Rerank:0.871
   患者: 女/50岁 | 证型: 肝亢肾虚
   摘要: 主要症状为失眠、多梦、早醒、烘热汗出、腰酸痛、尿频等；证型为肝亢肾虚；核心治法为益肾平肝、解郁安神；疗效显著，睡眠明显改善，伴随症状缓解。...


查询: 女性更年期失眠怎么办？
过滤: {'sex': '女', 'age_min': 40, 'age_max': 55}
召回 5 个医案:

1. [上海名老中医医案精选.md#20] 女/46岁 | 肝亢肾虚
   Rerank分数:

In [ ]:
# ==================== 示例调用 ====================

def run(query: str):
    state = {"query": query, "parsed_query": {}, "retrieved_cases": [], "answer": "", "messages": []}
    out = agent.invoke(state)
    print("=" * 80)
    print("Q:", query)
    print("--- messages ---")
    for m in out["messages"]:
        print(m)
    print("--- answer ---")
    print(out["answer"])
    return out


run("我最近失眠多梦，心烦易怒，中医怎么治疗？")
run("50岁女性，绝经后失眠严重，烘热汗出，腰酸，怎么办？")

# AgentState 工作原理（通俗解释）

`AgentState` 可以理解成 **Agent 在流水线里传递的一张“工作表”**。每跑完一个节点，就在这张表上“填几栏信息”，交给下一个节点继续用。

它包含这些字段：

- `query`
  - 用户原始问题。
  - 例："50岁女性绝经后失眠严重怎么办？"

- `parsed_query`
  - **把用户问题翻译成检索语言**后的结果。
  - 里面通常有：
    - `filters`：硬过滤条件（性别、年龄段、证型关键词等）
    - `rewritten_query`：为了向量检索更准，对问题做的“检索改写”

- `retrieved_cases`
  - RAG 检索得到的候选医案列表（已经做过向量召回 + rerank 排序）。

- `answer`
  - 最终要给用户的回答文本。

- `messages`
  - **流水线日志**。每个节点会往里面追加一句话，帮助你看清 agent 每一步到底做了什么。

## 3 个节点分别干什么

### 1) `parse` 节点（parse_query_node）
把自然语言问题变成“检索参数”。

- 输入：`query`
- 输出：写入 `parsed_query`

通俗理解：
- 用户说“50岁女性绝经后失眠、烘热汗出”
- 这个节点会提取：
  - 过滤条件：`sex=女`，`age_min=45`，`age_max=55`（如果能推断）
  - 改写查询：把口语变成更像“关键词+症状+证候”的检索句子

### 2) `retrieve` 节点（retrieve_cases_node）
真正去数据库里“找医案”。

- 输入：`parsed_query`
- 执行：
  - 先用 `rewritten_query` 生成 embedding（并截断到 `TARGET_DIM`，与入库一致）
  - pgvector 做向量相似度召回（可叠加 filters）
  - reranker 再对召回结果重排
- 输出：写入 `retrieved_cases`

通俗理解：
- 先“粗找”：向量找相似医案
- 再“精排”：reranker 把最相关的排前面

### 3) `generate` 节点（generate_answer_node）
把检索到的医案摘要当作“参考资料”，让 LLM 生成回答。

- 输入：`query` + `retrieved_cases`
- 输出：写入 `answer`

通俗理解：
- 这个节点不再“凭空回答”，而是：
  - 先把 top3 医案拼成上下文
  - 再让 LLM 结合这些医案去总结：证型、治法、用药方向

---

如果你希望 `parse` 不要“过度过滤”（导致检索 0 条），我可以在 `parse_query_node` 里加一个策略：
- **filters 只抽取 sex/age，不强行给 tcm_pattern**；
- 或者 `tcm_pattern` 只有在用户明确说了某证型才填。

# 运行顺序

1. 运行 `统一配置`
2. 运行 `LLM 结构化抽取`
3. 运行 `建表 + 入库（前 N 条）`
4. 运行 `RAG 检索`
5. 运行 `LangGraph Agent`
6. 运行 `示例调用`

如果你遇到 LLM 超时：
- 优先降低 `MAX_WORKERS` 到 1-2
- 或提高 `LLM_TIMEOUT_S`
- 或把 `content[:5000]` 调小（比如 3000）